# Trustworthy AI Explainer Dashboard: Interactive LLM-based Tutor with Explainability and Feedback
**Module 15 Team Project**

---

**Team Name:** Group4

**Team Members:**
1. Member A (PM/Integrator)
2. Member B (Gradio)
3. Member C (Streamlit)
4. Member D (Explainability)
5. Member E (Deployment/Docs)

**Institution:** CITEDI

**Date:** 05 Feb 2026

---

## Project overview

**Note: This is a team project. All team members should collaborate on all sections.**

This team project focuses on building user interfaces for LLM applications using Gradio and Streamlit. Your team will:

1. **UI framework understanding** (Gradio vs Streamlit) with clear trade-offs
2. **Gradio prototype** (chat + memory) deployed on Hugging Face Spaces
3. **Streamlit dashboard** deployed on Streamlit Community Cloud
4. **Chatbot interface** powered by OpenAI API with conversational memory
5. **Explainability integration** using **SHAP (global)** + **LIME (local)** alongside LLM outputs
6. **Feedback mechanism** for users to rate/flag outputs (logged to CSV)
7. **State & caching management** for performance and stable UX
8. **Production deployment** with secure secrets management (no hardcoded keys)

**Estimated completion time:** 80 minutes (per team member)

**Note:** This notebook contains learning materials and documentation. Final deliverables are the Python apps:
- `Group4_Module15_GradioApp.py` (Gradio / Hugging Face)
- `Group4_Module15_StreamlitApp.py` (Streamlit / Streamlit Cloud)


## Team planning and role assignment

**Team discussion: Decide how your team will divide the work and collaborate.**

**Suggested roles (adapt as needed):**
- **Gradio developer:** Lead rapid prototyping and Gradio interface development
- **Streamlit architect:** Design and implement Streamlit dashboard structure
- **Backend integrator:** Connect LLM, explainability, and feedback systems
- **Deployment specialist:** Handle deployment, testing, and documentation

**Your team's role distribution:**

| Team Member | Primary Role | Secondary Responsibilities |
|------------|--------------|----------------------------|
| Member A | System Architect & LLM Integration Lead | Defines LLM workflow, memory strategy, prompt policy, error handling |
| Member B | Explainability & Evaluation Lead (SHAP/LIME) | Designs quality signals, validates explainability outputs, test plans |
| Member C | Gradio Prototype Developer | Implements Gradio UI, integrates explanation panel, local testing |
| Member D | Streamlit Dashboard Architect | Implements Streamlit pages/state/caching, dashboard UX, local testing |
| Member E | Deployment & Documentation Specialist | GitHub repo hygiene, HF/Streamlit deploy, secrets, README/notebook/URLs |

**Application focus:** Tutor/Explainer chatbot on **Trustworthy AI**, aligned with prior modules (chat + memory + explainability + feedback).

**Target users:** University students, teaching staff, and institutional support personnel seeking trustworthy AI guidance and evaluation patterns.

**Collaboration approach:** GitHub (version control + deployments), WhatsApp (coordination), VS Code (local development & testing).


## Step 1: Environment setup

Install required libraries for UI development, LLM integration, and explainability.

**Note:** This notebook is for learning and documentation. Final deliverables are:
1. `Group4_Module15_GradioApp.py` - Gradio prototype (HF Spaces)
2. `Group4_Module15_StreamlitApp.py` - Streamlit dashboard (Streamlit Cloud)

Templates were adapted, tested locally in VS Code, and deployed with secrets management.



In [ ]:
# Install required libraries (uncomment to install)
# !pip install gradio streamlit
# !pip install openai langchain langchain-openai langchain-community
# !pip install pandas plotly
# !pip install shap lime  # For explainability
# !pip install sentence-transformers  # For embeddings

# Import libraries
import os
import time
from typing import List, Dict, Tuple, Optional

print("✅ Libraries ready!")
print("\nUI Frameworks:")
print("  - Gradio: For rapid prototyping")
print("  - Streamlit: For dashboards")
print("\nLLM Options:")
print("  - OpenAI API")
print("  - AWS Bedrock")
print("  - Local Ollama")
print("  - Hugging Face Transformers")
print("\nExplainability:")
print("  - SHAP (global)")
print("  - LIME (local)")

print("\n Using OpenAI API via environment/secrets in deployed apps (HF + Streamlit Cloud)")


## Part 1: Gradio rapid prototyping

### Understanding Gradio

**What is Gradio?**

Gradio is a Python library for quickly creating web interfaces for your machine learning models and functions.

**Key advantages:**
- Wrap any Python function in a web UI (minutes, not hours)
- Built-in components for ML tasks (chat, file upload, etc.)
- Native integration with Hugging Face Spaces
- Share via public link instantly
- Perfect for demos and proof-of-concepts

**Architecture pattern:**
```
Your Python Function → gr.Interface(fn=...) → Web Application
```

**When to use Gradio:**
- Quick prototypes and demos
- Sharing single functions/models
- PoC for stakeholders
- Hugging Face deployment

**Team note:** We implemented the full Gradio prototype and deployed it to Hugging Face Spaces with secure secrets.


In [ ]:
# Gradio Example 1: Simple Interface
import gradio as gr

def text_analyzer(text: str, analysis_type: str) -> dict:
    """Analyze text and return statistics."""
    return {
        "Character count": len(text),
        "Word count": len(text.split()),
        "Analysis type": analysis_type,
        "Sample": text[:50] + "..." if len(text) > 50 else text
    }

# Create Gradio interface
demo_simple = gr.Interface(
    fn=text_analyzer,
    inputs=[
        gr.Textbox(label="Enter text", lines=5, placeholder="Type or paste text here..."),
        gr.Radio(choices=["Basic", "Advanced"], label="Analysis Type", value="Basic")
    ],
    outputs=gr.JSON(label="Results"),
    title="Text Analyzer Demo",
    description="Simple Gradio interface demonstrating input/output mapping"
)

print("✅ Simple Gradio interface created!")
print("To run: demo_simple.launch()")
print("\n💡 Key insight: Gradio wraps your function with minimal code!")


### Gradio chatbot interface

**gr.ChatInterface** - Pre-built component for conversational AI

**Key features:**
- Automatically manages conversation history
- Built-in retry, undo, clear buttons
- Example prompts for users
- Streaming support
- Custom styling

**Function signature:**
```python
def chat_fn(message: str, history: List[Tuple[str, str]]) -> str:
    # message: current user message
    # history: list of (user_msg, bot_msg) tuples
    # return: bot response
```


In [ ]:
# Gradio Example 2: Chatbot with Memory
import gradio as gr
import time

def chatbot_response(message: str, history: list) -> str:
    """
    Generate chatbot response with context awareness.
    
    Args:
        message: Current user message
        history: List of (user_msg, bot_msg) tuples
    
    Returns:
        Bot response string
    """
    # Simulate processing time
    time.sleep(0.5)
    
    # Context-aware response
    num_exchanges = len(history)
    
    # Mock LLM response (replace with actual LLM)
    if "hello" in message.lower():
        response = f"Hello! This is exchange #{num_exchanges + 1}."
    elif "history" in message.lower():
        response = f"We've had {num_exchanges} exchanges so far."
    elif num_exchanges > 0:
        last_topic = history[-1][0][:30]
        response = f"You said: '{message}'. Earlier you asked about: '{last_topic}...'"
    else:
        response = f"You said: '{message}'. How can I help?"
    
    return response

# Create chatbot interface
chat_demo = gr.ChatInterface(
    fn=chatbot_response,
    title="Chatbot with Memory Demo",
    description="Try asking about the conversation history!",
    examples=["Hello!", "What did I say before?", "Tell me about history"],
    retry_btn="🔄 Retry",
    undo_btn="↩️ Undo",  
    clear_btn="🗑️ Clear History"
)

print("✅ Gradio chatbot created!")
print("To run: chat_demo.launch()")
print("\n💡 History is automatically managed by Gradio!")


## Part 2: Streamlit dashboard development

### Understanding Streamlit

**What is Streamlit?**

Streamlit turns Python scripts into interactive web applications.

**Key differences from Gradio:**

| Feature | Gradio | Streamlit |
|---------|---------|-----------|
| **Focus** | Function wrapping | Full applications |
| **Complexity** | Simple demos | Complex dashboards |
| **Layout** | Limited | Highly flexible |
| **State** | Implicit | Explicit (`session_state`) |
| **Best for** | Quick prototypes | Production apps |

**Streamlit's reactive model:**
1. User interacts (button, input, etc.)
2. Entire script re-runs top-to-bottom
3. UI updates with new values
4. State persists via `st.session_state`

**Critical concept: Caching**

Since scripts re-run on every interaction, expensive operations must be cached:
- `@st.cache_resource`: For models, connections (non-serialized objects)
- `@st.cache_data`: For data loading, computations (serializable data)


In [ ]:
# Streamlit Example: State Management Pattern
# Note: This is pseudo-code for learning - see streamlit_app_template.py for working code

streamlit_state_example = """
import streamlit as st

# Initialize session state (runs once per session)
if 'counter' not in st.session_state:
    st.session_state.counter = 0

if 'messages' not in st.session_state:
    st.session_state.messages = []

# Display current state
st.write(f"Counter: {st.session_state.counter}")
st.write(f"Messages: {len(st.session_state.messages)}")

# Buttons that modify state
if st.button("Increment Counter"):
    st.session_state.counter += 1
    st.rerun()  # Trigger re-run with new state

# Text input
if message := st.text_input("Add message"):
    st.session_state.messages.append(message)
    # State automatically persists

# Display all messages
for i, msg in enumerate(st.session_state.messages):
    st.write(f"{i+1}. {msg}")
"""

print("📝 Streamlit State Management Pattern:")
print(streamlit_state_example)
print("\n✅ Key concepts:")
print("  - Initialize state on first run")
print("  - Access via st.session_state")
print("  - State persists across reruns")
print("  - Use st.rerun() when needed")


### Streamlit caching strategies

**Why caching matters:**

Without caching, expensive operations run on every user interaction:
- Loading models (10+ seconds)
- Computing embeddings (seconds per call)
- API calls (cost + latency)
- Database queries

**Two caching decorators:**

**@st.cache_resource** - For non-serializable objects
```python
@st.cache_resource
def load_model():
    return HuggingFaceModel("model-name")  # Loads once, reused forever
```

Use for:
- ML models
- Database connections
- API clients
- Thread pools

**@st.cache_data** - For serializable data
```python
@st.cache_data(ttl=3600)  # Cache for 1 hour
def load_data():
    return pd.read_csv("data.csv")  # Cached by input params
```

Use for:
- DataFrames
- Lists, dicts
- Computation results
- API responses (with TTL)


In [ ]:
# Streamlit Caching Example
caching_example = """
import streamlit as st
import time

# BAD: Without caching (runs every time)
def expensive_computation_bad(x):
    time.sleep(3)  # Simulates expensive operation
    return x * 2

# GOOD: With caching (runs once per unique input)
@st.cache_data
def expensive_computation_good(x):
    time.sleep(3)  # Only runs first time for each x
    return x * 2

# Model loading example
@st.cache_resource
def load_llm():
    # Loads once, cached forever
    print("Loading model...")  # Only prints once
    return MockLLM()

# Usage
st.title("Caching Demo")

# First call: takes 3 seconds
# Subsequent calls with same input: instant
number = st.number_input("Enter number", value=5)
result = expensive_computation_good(number)
st.write(f"Result: {result}")

# Model loads once for entire session
model = load_llm()
"""

print("📝 Streamlit Caching Example:")
print(caching_example)
print("\n✅ Performance impact:")
print("  - Without caching: 3s per interaction")
print("  - With caching: 3s first time, <1ms after")
print("  - 3000x speedup!")


## Part 3: Team project implementation

### Your deliverables

Your team will create two complete applications:

**1. Gradio Prototype (`gradio_app_template.py`)**
- Simple chatbot interface
- LLM integration
- Basic explainability display
- User feedback mechanism (thumbs up/down)
- Ready to deploy to Hugging Face Spaces

**2. Streamlit Dashboard (`streamlit_app_template.py`)**
- Multi-page application:
  - Page 1: Chat interface with memory
  - Page 2: Explainability analysis
  - Page 3: Feedback dashboard
  - Page 4: System monitoring
  - Page 5: Documentation
- State management for conversation history
- Caching for expensive operations
- Feedback collection and visualization
- Performance metrics tracking

### Implementation guidance

**Templates provided:**

Both application templates are provided in your project directory with:
- Complete working code structure
- Placeholder functions for LLM integration
- Mock implementations for testing
- Detailed comments and TODOs
- Deployment instructions

**Your team should:**

1. **Customize the templates** for your use case
2. **Integrate actual LLM** (OpenAI, Bedrock, Ollama, or Transformers)
3. **Implement explainability** (SHAP or LIME visualizations)
4. **Test thoroughly** with different inputs and edge cases
5. **Deploy** to appropriate platform (Hugging Face Spaces or Streamlit Cloud)
6. **Document** your customizations and decisions


In [ ]:
# Verify template files exist
# Verify deliverable app files exist (Group_4 structure)
import os

deliverable_files = [
    "../apps/gradio/Group4_Module15_GradioApp.py",
    "../apps/streamlit/Group4_Module15_StreamlitApp.py",
    "../requirements.txt",
    "../README_Group4.md",
    "../deployment_urls.txt",
]

print("🔍 Checking for deliverable files...\n")

for file in deliverable_files:
    if os.path.exists(file):
        size = os.path.getsize(file)
        print(f"✅ {file} (Found: {size:,} bytes)")
    else:
        print(f"❌ {file} NOT FOUND")

print("\n📝 To run the applications locally:")
print("  Gradio:    python apps/gradio/Group4_Module15_GradioApp.py")
print("  Streamlit: streamlit run apps/streamlit/Group4_Module15_StreamlitApp.py")
print("\n💡 Deployed versions are available via deployment_urls.txt")


🔍 Checking for deliverable files...

✅ ../apps/gradio/Group4_Module15_GradioApp.py (Found: 19,645 bytes)
✅ ../apps/streamlit/Group4_Module15_StreamlitApp.py (Found: 25,599 bytes)
✅ ../requirements.txt (Found: 515 bytes)
✅ ../README_Group4.md (Found: 7,213 bytes)
✅ ../deployment_urls.txt (Found: 316 bytes)

📝 To run the applications locally:
  Gradio:    python apps/gradio/Group4_Module15_GradioApp.py
  Streamlit: streamlit run apps/streamlit/Group4_Module15_StreamlitApp.py

💡 Deployed versions are available via deployment_urls.txt


## Part 4: Deployment guide

### Deployment options

**1. Hugging Face Spaces (Gradio)**

Best for: Quick demos, public sharing

```bash
# Setup
git init
echo "gradio\nopenai" > requirements.txt
git add gradio_app_template.py requirements.txt
git commit -m "Initial commit"

# Deploy
# 1. Create new Space on huggingface.co
# 2. Choose "Gradio" as SDK
# 3. Push to the Space repo
git remote add hf https://huggingface.co/spaces/YOUR_USERNAME/YOUR_SPACE
git push hf main
```

**Important:** Use Hugging Face Secrets for API keys!

**2. Streamlit Community Cloud**

Best for: Team dashboards, internal tools

```bash
# Setup
git init  
echo "streamlit\npandas\nplotly" > requirements.txt
git add streamlit_app_template.py requirements.txt
git commit -m "Initial commit"
git push origin main

# Deploy
# 1. Go to share.streamlit.io
# 2. Connect your GitHub repo
# 3. Select branch and file
# 4. Deploy!
```

**Important:** Add secrets in Streamlit Cloud settings!

**3. Local Docker**

Best for: Development, self-hosted

```dockerfile
# Dockerfile for Streamlit
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY streamlit_app_template.py .
EXPOSE 8501
CMD ["streamlit", "run", "streamlit_app_template.py"]
```

```bash
docker build -t my-ai-dashboard .
docker run -p 8501:8501 my-ai-dashboard
```

### Security best practices

**Never hardcode API keys:**

```python
# WRONG
openai.api_key = "sk-abc123..."

# RIGHT - Environment variables
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

# RIGHT - Streamlit secrets
import streamlit as st
openai.api_key = st.secrets["OPENAI_API_KEY"]
```

**Secrets configuration:**

**Hugging Face Spaces:**
- Settings → Repository secrets
- Add `OPENAI_API_KEY = your_key`
- Access via `os.getenv()`

**Streamlit Cloud:**
- App settings → Secrets
- Add TOML format:
```toml
OPENAI_API_KEY = "your_key"
```
- Access via `st.secrets["OPENAI_API_KEY"]`


**Team deployment used:** Hugging Face Spaces (Gradio) + Streamlit Community Cloud (Streamlit) + GitHub for source control.



## Team project summary

### A. Team information

**Team Name:** Group_4

**Application Name:** Trustworthy AI Tutor/Explainer Dashboard

**Target Use Case:** Interactive LLM-based tutor/explainer for Trustworthy AI concepts with built-in explainability (SHAP/LIME), feedback logging, and basic monitoring.

**Target Users:** University students, teaching staff, and institutional support personnel.

### B. Gradio prototype summary

**Completed features:**
- [x] Basic interface with inputs/outputs
- [x] LLM integration
- [x] Chatbot with conversation history
- [x] Explainability display
- [x] User feedback mechanism
- [x] Deployed to Hugging Face Spaces

**LLM choice:** OpenAI API (OpenAI Python SDK)

**Key customizations made:** Chat with memory, integrated Explainability panel (quality score + SHAP + LIME), user feedback logging to CSV, safe secrets handling (no hardcoded keys), mock fallback when key missing.

**Deployment URL:** https://huggingface.co/spaces/alej-hugg/group-4-module15-gradio

### C. Streamlit dashboard summary

**Completed features:**
- [x] Multi-page structure
- [x] Chat interface with memory
- [x] State management for history
- [x] Caching for expensive operations
- [x] Explainability analysis page
- [x] Feedback collection and visualization
- [x] Performance monitoring
- [x] Documentation page
- [x] Deployed to Streamlit Cloud

**Pages implemented:**
1. Chat (LLM + memory)
2. Explainability (quality signals + SHAP/LIME summary)
3. Feedback (CSV logging + basic review)
4. Monitoring (simple runtime signals / counters)
5. About/Docs (how-to + limitations)

**State management approach:** `st.session_state` for conversation history and UI state; Gradio uses component state/history and message conversion utilities. No cross-user leakage (per-session state).

**Caching strategy:** `@st.cache_resource` for the OpenAI client / model-like resources; lightweight computations cached where appropriate to reduce re-run overhead.

**Deployment URL:** https://group-4-module15-strmlit.streamlit.app/

### D. Explainability integration

**Method used:** [x] SHAP [x] LIME [x] Both

**Implementation details:** We compute a lightweight *quality score (0–1)* from observable output signals (e.g., presence of steps/structure, uncertainty markers, length control) and then use SHAP to explain which signals contributed most to the score. We also use LIME locally to highlight which user-input tokens are most influential for the produced response in that specific interaction.

**Visualizations created:** Explainability summary (quality score + signals), SHAP top feature contributions table, LIME influential user tokens list.

**User value provided:** Users can understand “why” an answer is scored as higher quality and what parts of their prompt are steering the response, supporting transparency and iterative prompt improvement.

### E. Feedback mechanism

**Feedback type:** Thumbs up/down (and optional flags), captured per response.

**Storage approach:** CSV logging inside the app runtime (prototype). In production, migrate to persistent DB/storage.

**Analytics implemented:** Basic feedback counts and review via dashboard page; logs available for manual inspection and future aggregation.

**Total feedback collected (during testing):** 5–15 (during structured test prompts; may vary by run)

### F. Team collaboration

**Role distribution:**

| Team Member | Primary Role | Key Contributions |
|------------|--------------|-------------------|
| Member A | System Architect & LLM Integration Lead | LLM workflow, memory strategy, prompt policy, error handling |
| Member B | Explainability & Evaluation Lead (SHAP/LIME) | Quality signals, SHAP/LIME integration, testing protocol |
| Member C | Gradio Prototype Developer | Gradio UI, explainability panel, local tests, HF readiness |
| Member D | Streamlit Dashboard Architect | Pages/state/caching, UX, local tests, Cloud readiness |
| Member E | Deployment & Documentation Specialist | GitHub, secrets, deployments, README, URLs, notebook completion |

**Collaboration tools used:** GitHub, WhatsApp, VS Code.

**Challenges encountered:** Secrets/env loading differences between local, HF Spaces, and Streamlit Cloud; Gradio message formatting differences across versions; stable state handling across reruns in Streamlit.

**Solutions implemented:** Platform-specific secrets usage (HF Secrets / Streamlit Secrets), robust fallback to mock mode, history normalization utilities for Gradio, session_state patterns and caching for Streamlit.

### G. Technical decisions

**Framework comparison insights:**
- Gradio is optimal for fast demos and single-purpose UIs; Streamlit is optimal for structured dashboards and multi-page applications.

**When to use Gradio:**
- Rapid prototyping and demos (minutes to UI)
- Hugging Face Spaces deployment for public sharing

**When to use Streamlit:**
- Multi-page dashboards with monitoring/feedback views
- Fine-grained layout control, state and caching for “app-like” behavior

**Most valuable feature learned:** Managing state + explainability side-by-side with LLM outputs (transparent UX).

**Most challenging aspect:** Ensuring consistent secrets/config behavior across deployment platforms while keeping the code secure.

### H. Deployment experience

**Platform(s) used:** Hugging Face Spaces (Gradio) + Streamlit Community Cloud (Streamlit)

**Deployment challenges:** Correct file paths, secrets configuration, and ensuring the app runs outside local `.env` environments.

**Security measures implemented:** No hardcoded secrets; API keys stored only in HF Secrets and Streamlit Secrets; `.env` kept local and excluded from Git tracking.

**Performance observations:** Streamlit caching improved responsiveness by avoiding reinitialization on every interaction; overall latency dominated by LLM API calls.

### I. Future improvements

**Short-term enhancements:**
1. Replace CSV with persistent storage (SQLite/Postgres) for feedback
2. Add standardized evaluation metrics (rubric-based scoring, safety checks)
3. Add better observability (structured logs, tracing, error dashboards)

**Long-term vision:**
A production-grade trustworthy tutoring platform with robust evaluation, auditing, and monitoring for institutional use.

**Scalability considerations:**
Rate limiting, caching, batching, persistent storage, and separation of UI from backend services.

### J. Learning outcomes

**Key concepts mastered:**
- [x] Gradio interface development
- [x] Streamlit dashboard creation
- [x] State management in reactive apps
- [x] Caching strategies for performance
- [x] Multi-page application architecture
- [x] Deployment to cloud platforms
- [x] API key security best practices
- [x] User feedback collection
- [x] Explainability integration

**Most valuable insight:** Trustworthy AI UX is not only model choice—it's also explainability, feedback loops, and deployable interfaces with secure configuration.

**How this applies to real projects:** The same pattern generalizes to institutional chatbots (policies, student support, compliance) where transparency, evaluation, and monitoring are mandatory.

**Skills gained:** UI prototyping (Gradio), dashboard engineering (Streamlit), secrets-based deployments, and practical explainability integration for LLM applications.



## Team submission checklist

Before submitting, ensure your team has completed:

**Planning and setup:**
- [X] Team roles clearly defined and documented
- [X] Application use case and target users identified
- [X] Development environment configured
- [X] All required libraries installed

**Gradio prototype:**
- [X] `gradio_app_template.py` customized for your use case
- [X] LLM integrated (OpenAI/Bedrock/Ollama/Transformers)
- [X] Chatbot interface functional with conversation memory
- [X] Explainability display implemented
- [X] User feedback mechanism (thumbs up/down) working
- [X] Tested with multiple example conversations
- [X] Deployed to Hugging Face Spaces (or local deployment documented)
- [X] API keys properly secured (no hardcoded secrets)

**Streamlit dashboard:**
- [X] `streamlit_app_template.py` customized for your use case
- [X] Multi-page structure implemented
- [X] Chat page with conversation history working
- [X] State management implemented for persistence
- [X] Caching applied to expensive operations
- [X] Explainability analysis page functional
- [X] Feedback dashboard with visualizations
- [X] Monitoring page showing metrics
- [X] Documentation page completed with team information
- [X] Tested thoroughly (state, caching, all pages)
- [X] Deployed to Streamlit Cloud (or local deployment documented)
- [X] API keys secured via secrets management

**Explainability integration:**
- [X] SHAP or LIME implemented
- [X] Visualizations display correctly
- [X] Explanations integrated with LLM responses
- [X] Users can understand model decisions

**Documentation:**
- [X] Team information documented
- [X] Role distribution explained
- [X] Technical decisions justified
- [X] Deployment instructions provided
- [X] User guide included
- [X] Code well-commented

**Testing:**
- [X] Both applications tested with various inputs
- [X] Edge cases handled (empty input, long text, errors)
- [X] Performance acceptable (caching working)
- [X] Feedback mechanism collecting data properly
- [X] Deployment working (accessible URLs)

**Submission files:**
- [X] `Group4_Module15_Project.ipynb` (this completed notebook)
- [X] `Group4_Module15_GradioApp.py` (customized Gradio app)
- [X] `Group4_Module15_StreamlitApp.py` (customized Streamlit app)
- [X] `requirements.txt` (all dependencies listed)
- [X] `README_Group4.md` (deployment and usage instructions)
- [X] `deployment_urls.txt` (links to deployed applications)

**Estimated completion time: 80 minutes (per team member)**

---

## Resources and next steps

**Official documentation:**
- Gradio: https://www.gradio.app/docs
- Streamlit: https://docs.streamlit.io/
- Hugging Face Spaces: https://huggingface.co/docs/hub/spaces
- Streamlit Community Cloud: https://streamlit.io/cloud

**Deployment platforms:**
- Hugging Face Spaces: https://huggingface.co/spaces
- Streamlit Cloud: https://share.streamlit.io
- Docker Hub: https://hub.docker.com/

**Explainability libraries:**
- SHAP: https://shap.readthedocs.io/
- LIME: https://lime-ml.readthedocs.io/

**LLM integration:**
- OpenAI: https://platform.openai.com/docs
- LangChain: https://python.langchain.com/docs
- Hugging Face Transformers: https://huggingface.co/docs/transformers

---

**End of Module 15 Team Project**

**Good luck with your Trustworthy AI Explainer Dashboard!** 
